# WebScraping - Site : L'Equipe - liste & caractéristiques des joueurs

## Script  avec la librairie Selenium

-> avoir la liste de tous les joueurs par équipe - Coupe du monde 2018

In [1]:
import lxml
from time import sleep
from bs4 import BeautifulSoup as bs
from selenium import webdriver
import csv
import pandas as pd

In [2]:
#Create an chrome options object
chrome_options = webdriver.ChromeOptions()
#Create the option to disable notification
prefs = {"profile.default_content_setting_values.notifications" : 2}
#Add this option to my options
chrome_options.add_experimental_option("prefs",prefs)
#Create a driver with my options
driver = webdriver.Chrome(chrome_options=chrome_options)

In [3]:
#Store Footballpage url
footurl="https://www.lequipe.fr/Football/EQ_CM.html"
#Open the homepage on the driver
driver.get(footurl)
sleep(0.5)

In [4]:
#Scroll the page to the end
for i in range(0,15):
    driver.execute_script("window.scrollTo(0,999999999999)")
    sleep(0.5)

In [5]:
#Get the source code and parse
page = driver.page_source
soup = bs(page, 'lxml')

In [6]:
#Find the container with all teams informations
wrapper = soup.find('div', {'id': 'Base'})

In [ ]:
#wrapper

In [7]:
# Initialize table to store countries'name & countries'url
club_url=[]

In [8]:
#Collect all "effectif" url
effectifs = wrapper.find_all('a')
for link in effectifs:
    link=link.get('href')
    if link[len(link)-1] == "F": #select the last letter of the url
        club_url.append("https://www.lequipe.fr/Football/"+link) #insert teams url into the table(club_url)

In [9]:
print(club_url)

['https://www.lequipe.fr/Football/FootballFicheSelection159.html#EFF', 'https://www.lequipe.fr/Football/FootballFicheSelection170.html#EFF', 'https://www.lequipe.fr/Football/FootballFicheSelection631.html#EFF', 'https://www.lequipe.fr/Football/FootballFicheSelection642.html#EFF', 'https://www.lequipe.fr/Football/FootballFicheSelection1889.html#EFF', 'https://www.lequipe.fr/Football/FootballFicheSelection644.html#EFF', 'https://www.lequipe.fr/Football/FootballFicheSelection626.html#EFF', 'https://www.lequipe.fr/Football/FootballFicheSelection640.html#EFF', 'https://www.lequipe.fr/Football/FootballFicheSelection636.html#EFF', 'https://www.lequipe.fr/Football/FootballFicheSelection5000000000000000000001351.html#EFF', 'https://www.lequipe.fr/Football/FootballFicheSelection187.html#EFF', 'https://www.lequipe.fr/Football/FootballFicheSelection151.html#EFF', 'https://www.lequipe.fr/Football/FootballFicheSelection654.html#EFF', 'https://www.lequipe.fr/Football/FootballFicheSelection175.html#EF

In [10]:
#Create DF
df_joueurs = pd.DataFrame(columns=['noms_pays','nom_joueur', 'poste', 'taille', 'poids', 'nb_sel'])



In [11]:
#parse each url in club_url
for url in club_url :
    driver.get(url)
    sleep(0.5)
    driver.find_element_by_css_selector("#EFF .seemore").click() #click on the "seemore"
    sleep(0.5)
    for i in range(0,5): #scroll the list (5 times)
        driver.execute_script("window.scrollTo(0,9999999999999999)")
        sleep(0.5)
    page = driver.page_source #get the source_code
    soup = bs(page, 'lxml')
    wrapper = soup.find('table', {'id': 'club_effectif'})
    table = wrapper.find_all("tr")
    for r in table:
        playerinfos = [] #initialize a table
        td = r.find_all("td")

        for data in td:
            playerinfos.append(data.text) #collect informations
            try:
                df_joueurs = df_joueurs.append({'noms_pays':playerinfos[0],'nom_joueur':playerinfos[1],'poste':playerinfos[3], 'taille':playerinfos[4], 'poids':playerinfos[5], 'nb_sel':playerinfos[7]}, ignore_index=True)
            except IndexError:
                pass
print(df_joueurs)

     noms_pays               nom_joueur poste taille poids    nb_sel
0                         Thomas Müller  3MIL  1.86m  74kg   94 sel.
1                         Thomas Müller  3MIL  1.86m  74kg   94 sel.
2                            Mesut Özil  3MIL  1.82m  70kg   92 sel.
3                            Mesut Özil  3MIL  1.82m  70kg   92 sel.
4                            Toni Kroos  3MIL  1.82m  77kg   86 sel.
5                            Toni Kroos  3MIL  1.82m  77kg   86 sel.
6                          Manuel Neuer  1GAR  1.93m  90kg   79 sel.
7                          Manuel Neuer  1GAR  1.93m  90kg   79 sel.
8                           Mario Gomez  4ATT  1.89m  88kg   78 sel.
9                           Mario Gomez  4ATT  1.89m  88kg   78 sel.
10                         Sami Khedira  3MIL  1.89m  81kg   77 sel.
11                         Sami Khedira  3MIL  1.89m  81kg   77 sel.
12                       Jerome Boateng  2DEF  1.92m  90kg   73 sel.
13                       Jerome Bo

In [12]:
df_joueurs = df_joueurs.drop_duplicates(keep='first')

In [13]:
df_joueurs

,noms_pays,nom_joueur,poste,taille,poids,nb_sel
0,,Thomas Müller,3MIL,1.86m,74kg,94 sel.
2,,Mesut Özil,3MIL,1.82m,70kg,92 sel.
4,,Toni Kroos,3MIL,1.82m,77kg,86 sel.
6,,Manuel Neuer,1GAR,1.93m,90kg,79 sel.
8,,Mario Gomez,4ATT,1.89m,88kg,78 sel.
10,,Sami Khedira,3MIL,1.89m,81kg,77 sel.
12,,Jerome Boateng,2DEF,1.92m,90kg,73 sel.
14,,Mats Hummels,2DEF,1.91m,88kg,66 sel.
16,,Julian Draxler,3MIL,1.87m,72kg,46 sel.
18,,Jonas Hector,2DEF,1.85m,73kg,40 sel.


In [14]:
df_joueurs.to_csv('joueurs_cdm_2018.csv', sep='\t', encoding='utf-8')

#### Après avoir nettoyer le csv, je le remets au format DF

In [15]:
df_joueurs_18 = pd.read_csv("Joueurs_cdm_18.csv",sep = ',')

In [16]:
df_joueurs_18

,noms_pays,nom_joueur,poste,taille,poids,nb_sel
0,Allemagne,Thomas Müller,MIL,1.86m,74kg,94 sel.
1,Allemagne,Mesut Özil,MIL,1.82m,70kg,92 sel.
2,Allemagne,Toni Kroos,MIL,1.82m,77kg,86 sel.
3,Allemagne,Manuel Neuer,GAR,1.93m,90kg,79 sel.
4,Allemagne,Mario Gomez,ATT,1.89m,88kg,78 sel.
5,Allemagne,Sami Khedira,MIL,1.89m,81kg,77 sel.
6,Allemagne,Jerome Boateng,DEF,1.92m,90kg,73 sel.
7,Allemagne,Mats Hummels,DEF,1.91m,88kg,66 sel.
8,Allemagne,Julian Draxler,MIL,1.87m,72kg,46 sel.
9,Allemagne,Jonas Hector,DEF,1.85m,73kg,40 sel.
